In [ ]:
import json
import pandas as pd

llama_paths = ['../data/results/evaluate_probes/results_0UAqFzWs.jsonl',
              '../data/results/evaluate_probes/results_8qeQijce.jsonl',
              '../data/results/evaluate_probes/results_48Y3tT3Q.jsonl',
              '../data/results/evaluate_probes/results_DK4FrUMp.jsonl']

gemma_paths = ['../data/results/evaluate_probes/results_Z0ttu4Ki.jsonl', # 27b
               '../data/results/evaluate_probes/results_PUZIiI9J.jsonl', # 1b
               '../data/results/evaluate_probes/results_iXWVMLiF.jsonl'] # 12b


paths = ['../data/results/evaluate_probes/results_9CfJYP0l_test.jsonl',
         '../data/results/evaluate_probes/results_j6GrF3LD_test.jsonl',
         '../data/results/evaluate_probes/results_oa3Vl8cy_test.jsonl',
         '../data/results/evaluate_probes/results_PI4AwFSc_test.jsonl',
         '../data/results/evaluate_probes/results_Q5HuZ0M5_test.jsonl',
         '../data/results/evaluate_probes/results_vikDTPcl_test.jsonl',
         '../data/results/evaluate_probes/best_probe.jsonl']

# Method 1: Using pure Python
data = []
for path in paths:
    with open(path, 'r') as file:
        for line in file:
            data.append(json.loads(line.strip()))

data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a DataFrame from the data
df_list = []
for item in data:
    df_list.append({
        'model_name': item['config']['model_name'],
        'dataset_name': item['dataset_name'],
        'auroc': item['metrics']['metrics']['auroc'],
        'max_samples': item['config']['max_samples']
    })

df = pd.DataFrame(df_list)
df = df[df['max_samples'].isnull()]

# Define the desired order of models
model_order = [
    'meta-llama/Llama-3.2-1B-Instruct',
    'meta-llama/Llama-3.2-3B-Instruct',
    'meta-llama/Llama-3.1-8B-Instruct',
    'meta-llama/Llama-3.3-70B-Instruct',
    'google/gemma-3-1b-it',
    'google/gemma-3-12b-it', 
    'google/gemma-3-27b-it'
]

# Create a categorical type with the custom order
df['model_name'] = pd.Categorical(df['model_name'], categories=model_order, ordered=True)

# Sort the dataframe by the ordered model names
df = df.sort_values('model_name')


# Set the style and figure size
plt.figure(figsize=(10, 6))
sns.set_style("whitegrid")

# Set the style parameters for black edges on bars
plt.rcParams['patch.edgecolor'] = 'black'
plt.rcParams['patch.force_edgecolor'] = True
plt.rcParams['patch.linewidth'] = 1


# Create the bar plot
sns.barplot(
    data=df,
    x='dataset_name',
    y='auroc',
    hue='model_name',
    palette='tab10'
)

# Customize the plot
#plt.title('Probe Performance on Different Scales of Models', fontsize=14, pad=20)
plt.xlabel('Dataset', fontsize=12)
plt.ylabel('AUROC', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.ylim(0.6, 1)  # AUROC ranges from 0 to 1


# Adjust legend
plt.legend(title='Model', loc ='lower left', ncol=2) #bbox_to_anchor=(1.05, 1), loc='upper left')

# Adjust layout to prevent label cutoff
plt.tight_layout()

# Save the figure as PDF
plt.savefig('../images/model_scale_comparison.pdf', format='pdf', bbox_inches='tight', dpi=300)

# Show the plot
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Create a DataFrame from the data
df_list = []
for item in data:
    df_list.append({
        'model_name': item['config']['model_name'],
        'dataset_name': item['dataset_name'],
        'auroc': item['metrics']['metrics']['auroc'],
        'max_samples': item['config']['max_samples']
    })

df = pd.DataFrame(df_list)
df = df[df['max_samples'].isnull()]

# Extract model size from model name
def get_model_size(model_name:str):
    if 'llama' in model_name.lower():
        if '1b' in model_name.lower():
            return 1
        elif '3b' in model_name.lower():
            return 3
        elif '8b' in model_name.lower():
            return 8
        elif '70b' in model_name.lower():
            return 70
    elif 'gemma' in model_name.lower():
        if '1b' in model_name.lower():
            return 1
        elif '12b' in model_name.lower():
            return 12
        elif '27b' in model_name.lower():
            return 27
    return None

# Add model size and model family columns
df['model_size'] = df['model_name'].apply(get_model_size)
df['model_family'] = df['model_name'].apply(lambda x: 'Llama' if 'llama' in x.lower() else 'Gemma')

# Calculate mean AUROC for each model
model_means = df.groupby(['model_name', 'model_size', 'model_family'])['auroc'].mean().reset_index().sort_values('model_size')

# Create the plot
plt.figure(figsize=(10, 6))
sns.set_style("whitegrid")

# Plot lines for each model family
for family in ['Llama', 'Gemma']:
    family_data = model_means[model_means['model_family'] == family]
    plt.plot(family_data['model_size'], family_data['auroc'], 
             marker='o', linewidth=2, label=family)

# Customize the plot
#plt.title('Mean AUROC Scores by Model Size', fontsize=14, pad=20)
plt.xlabel('Model Size (Billion Parameters)', fontsize=12)
plt.ylabel('Mean AUROC', fontsize=12)
#plt.xscale('log')  # Use log scale for model size
plt.ylim(0.75, 0.95)  # AUROC ranges from 0 to 1

# Add grid
plt.grid(True, alpha=0.3)

# Add legend
plt.legend(title='Model Family', loc='upper left')

# Adjust layout
plt.tight_layout()

# Save the figure as a PDF
plt.savefig('../images/scaling_plot.pdf', format='pdf', bbox_inches='tight', dpi=300)

# Show the plot
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Create a DataFrame from the data
df_list = []
for item in data:
    df_list.append({
        'model_name': item['config']['model_name'],
        'dataset_name': item['dataset_name'],
        'auroc': item['metrics']['metrics']['auroc'],
        'max_samples': item['config']['max_samples']
    })

df = pd.DataFrame(df_list)
df = df[df['max_samples'].isnull()]

# Define the desired order of models
model_order = [
    'meta-llama/Llama-3.2-1B-Instruct',
    'meta-llama/Llama-3.2-3B-Instruct',
    'meta-llama/Llama-3.1-8B-Instruct',
    'meta-llama/Llama-3.3-70B-Instruct',
    'google/gemma-3-1b-it',
    'google/gemma-3-12b-it', 
    'google/gemma-3-27b-it'
]

# Create a categorical type with the custom order
df['model_name'] = pd.Categorical(df['model_name'], categories=model_order, ordered=True)

# Sort the dataframe by the ordered model names
df = df.sort_values('model_name')

# Clean up model names by removing meta-llama/ and google/
df['model_name'] = df['model_name'].str.replace('meta-llama/', '')
df['model_name'] = df['model_name'].str.replace('google/', '')


# Extract model size from model name
def get_model_size(model_name:str):
    if 'llama' in model_name.lower():
        if '1b' in model_name.lower():
            return 1
        elif '3b' in model_name.lower():
            return 3
        elif '8b' in model_name.lower():
            return 8
        elif '70b' in model_name.lower():
            return 70
    elif 'gemma' in model_name.lower():
        if '1b' in model_name.lower():
            return 1
        elif '12b' in model_name.lower():
            return 12
        elif '27b' in model_name.lower():
            return 27
    return None

# Add model size and model family columns
df['model_size'] = df['model_name'].apply(get_model_size)
df['model_family'] = df['model_name'].apply(lambda x: 'Llama' if 'llama' in x.lower() else 'Gemma')

# Calculate mean AUROC for each model
#model_means = df.groupby(['model_name', 'model_size', 'model_family'])['auroc'].mean().reset_index().sort_values('model_size')

model_means = df.groupby(['model_name', 'model_size', 'model_family'])['auroc'].mean()#.reset_index().sort_values('model_size')
model_means = model_means[~model_means.isnull()].reset_index().sort_values('model_size')

# Create a figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7), gridspec_kw={'width_ratios': [6, 4]})
sns.set_style("whitegrid")

# Set the style parameters for black edges on bars
plt.rcParams['patch.edgecolor'] = 'black'
plt.rcParams['patch.force_edgecolor'] = True
plt.rcParams['patch.linewidth'] = 1

# First subplot: Bar plot
sns.barplot(
    data=df,
    x='dataset_name',
    y='auroc',
    hue='model_name',
    palette='tab10',
    ax=ax1
)

# Customize the first subplot
ax1.set_xlabel('Dataset', fontsize=12)
ax1.set_ylabel('AUROC', fontsize=12)
ax1.tick_params(axis='x', rotation=15)
ax1.set_ylim(0.6, 1)
ax1.legend(title='Model', loc='lower left', ncol=2, framealpha=1.0)


# Second subplot: Scaling plot
for family in ['Llama', 'Gemma']:
    family_data = model_means[model_means['model_family'] == family]
    ax2.plot(family_data['model_size'], family_data['auroc'], 
             marker='o', linestyle='-', linewidth=2, label=family)

# Customize the second subplot
ax2.set_xlabel('Model Size (Billion Parameters)', fontsize=12)
ax2.set_ylabel('Mean AUROC', fontsize=12)
ax2.set_ylim(0.77, 0.925)
ax2.grid(True, alpha=0.3)
ax2.legend(title='Model Family', loc='lower right')

# Adjust layout to prevent label cutoff
plt.tight_layout()

sns.set_context("notebook", font_scale=1.4)
# Set font scale for axis labels
ax1.xaxis.label.set_size(17)
ax1.yaxis.label.set_size(17)
ax2.xaxis.label.set_size(17)
ax2.yaxis.label.set_size(17)


# Save the figure as PDF
plt.savefig('../images/combined_scaling_plots.pdf', format='pdf', bbox_inches='tight', dpi=300)

# Show the plot
plt.show()

In [ ]:
model_means = df.groupby(['model_name', 'model_size', 'model_family'])['auroc'].mean()#.reset_index().sort_values('model_size')
model_means = model_means[~model_means.isnull()].reset_index().sort_values('model_size')
model_means